In [1]:
from IPython.display import display,HTML,clear_output
import json

# Read explanation report

In [2]:
explanation_report_filepath = "/Users/salvatore/PycharmProjects/T-EBAnO-Express/outputs/20201117_bert_model_imdb_reviews_exp_0/local_explanations_experiments/20201206_121122/local_explanations/local_explanation_report_4.json"

In [3]:
with open(explanation_report_filepath) as json_file:
    explanation_report = json.load(json_file)

# Parse explanation report

In [4]:
input_info = explanation_report["input_info"]
input_positions_tokens = input_info["positions_tokens"]
raw_text = input_info["raw_text"]
cleaned_text = input_info["cleaned_text"]
preprocessed_text = input_info["preprocessed_text"]

In [5]:
pos_local_explanations = []  # pos features without combinations
sen_local_explanations = []  # sen features without combinations
mlwe_local_explanations = []  # mlwe features without combinations
pos_local_explanations_comb = []  # All pos features (also combinations)
sen_local_explanations_comb = []  # All sen features (also combinations)
mlwe_local_explanations_comb = []  # All mlwe features (also combinations)

for le in explanation_report["local_explanations"]:
    if le["feature_type"] == "POS" and le["combination"] == 1:
        pos_local_explanations.append(le)
    if le["feature_type"] == "SEN" and le["combination"] == 1:
        sen_local_explanations.append(le)
    if le["feature_type"] == "MLWE" and le["combination"] == 1:
        mlwe_local_explanations.append(le)
    if le["feature_type"] == "POS":
        pos_local_explanations_comb.append(le)
    if le["feature_type"] == "SEN":  
        sen_local_explanations_comb.append(le)
    if le["feature_type"] == "MLWE":
        mlwe_local_explanations_comb.append(le)
        

In [6]:
pos_positions_tokens_score = {} # Dictionary for pos features with position as key and as value the tuple (token, nPIR)
sen_positions_tokens_score = {}#  Dictionary for sen features with position as key and as value the tuple (token, nPIR)
mlwe_positions_tokens_score = {} # Dictionary for mlwe features with position as key and as value the tuple (token, nPIR)

for le in pos_local_explanations:
    for position, token in le["positions_tokens"].items():
        pos_positions_tokens_score[position] = (token, round(le["nPIR_original_top_class"],4))
        
for le in sen_local_explanations:
    for position, token in le["positions_tokens"].items():
        sen_positions_tokens_score[position] = (token, round(le["nPIR_original_top_class"],4))

for le in mlwe_local_explanations:
    for position, token in le["positions_tokens"].items():
        mlwe_positions_tokens_score[position] = (token, round(le["nPIR_original_top_class"],4))
    

# Utility functions


In [7]:
 def label_name(label):
    return ["Negative","Positive"][label]

def get_summary_string(position_token_score):
    positive_color = "background-color:#00FF00"
    negative_color = "background-color:#FF00FF"
    html_string = ""

    for position in sorted(position_token_score.keys(), key=lambda k: int(k)):
        token_score = position_token_score[position]
        if token_score[1] >= 0:
            positive_color = "background-color:rgba(124, 252, 0, {})".format(token_score[1])
            html_string = html_string + '<span style="{}"><b>{}</b></span> '.format(positive_color,token_score[0])
        else:
            negative_color = "background-color:rgba(255, 99, 71, {})".format(token_score[1])
            html_string = html_string + '<span style="{}"><b>{}</b></span> '.format(negative_color,token_score[0])
            
    return html_string

def get_feature_colored_html_string(input_positions_tokens, feature_positions_tokens):
    html_string = ""
    feature_color = "background-color:rgba(0, 255, 255, 1)"
    for position in range(len(input_positions_tokens)):
        if str(position) in feature_positions_tokens:
            html_string = html_string + '<span style="{}"><b>{}</b></span> '.format(feature_color,input_positions_tokens[position])
        else:
            html_string = html_string + '<span><b>{}</b></span> '.format(input_positions_tokens[position])
    return html_string
    

def print_local_explanation(le, input_positions_tokens):
    if len(le["positions_tokens"]) == 0:
        return
    print("LocalExplanationId: {}, FeatureId: {}".format(le["local_explanation_id"], le["feature_id"]))
    print("feature type: ",le["feature_type"])
    print("feature description: ", le["feature_description"])
    print("feature (position,token): " ,le["positions_tokens"])
    print("Perturbed Text:  ",le["perturbed_text"])
    print("perturbed probabilities: ",le["perturbed_probabilities"])
    print("nPIR Original Class: " ,round(le["nPIR_original_top_class"],3))
    display(HTML(get_feature_colored_html_string(input_positions_tokens,le["positions_tokens"])))
    print("\n")
    print("\n")
    return

In [8]:
display(HTML('<h1>Input Text info</h1>'))
display(HTML('Raw Text:   <b>{} </b>'.format(raw_text)))
display(HTML('Clean Text:   <b>{} </b>'.format(cleaned_text)))
display(HTML('Preprocessed Text:   <b>{} </b>'.format(preprocessed_text)))

In [9]:
display(HTML('<h1>Multi-Layer Word Embedding Features Summary</h1>'))
display(HTML('Predicted Class: <b>{} </b>'.format(label_name(input_info["original_label"]))))
display(HTML('Original Probability: <b>{} </b>'.format(input_info["original_probabilities"])))

html_string = get_summary_string(mlwe_positions_tokens_score)
        
display(HTML(html_string))

display(HTML('(Features are highlighted in green if are positively influential for the predicted class, red otherwise. The intensity of the color is proportional to the nPIR score obtained by the word.) '))  


# MLWE Most influential feature

Feature with highest nPIR 

In [10]:
most_influnetial_mlwe_feature =max(mlwe_local_explanations_comb, key= lambda le: le["nPIR_original_top_class"])
print_local_explanation(most_influnetial_mlwe_feature,input_positions_tokens)

LocalExplanationId: 41, FeatureId: 3
feature type:  MLWE
feature description:  Combination of Cluster 0 and Cluster 1
feature (position,token):  {'2': 'this', '3': 'movie', '9': 'just', '10': 'for', '11': 'kicks', '12': 'decided', '13': 'to', '14': 'rent', '15': 'it', '16': 'and', '17': 'watch', '18': 'it', '21': 'the', '22': 'plot', '23': 'is', '24': 'a', '25': 'carbon', '26': 'copy', '27': 'from', '28': 'fright', '31': 'i', '32': 'did', '33': 'like', '43': 'otherwise', '44': 'it', '45': 'was', '46': 'not', '47': 'good', '48': 'at', '49': 'all', '29': 'night', '34': 'the', '35': 'hairy', '36': 'vampire', '37': 'and', '38': 'the', '39': 'bug', '40': 'eating', '41': 'driver'}
Perturbed Text:   i saw many years ago, and again....
perturbed probabilities:  [0.004752695560455322, 0.9952473044395447]
nPIR Original Class:  0.995


<h2>Query Multi-Layer Word Embedding features</h2>

In [11]:
print("Original Probabilities: {}".format(input_info["original_probabilities"]))
print("Original label: {}".format(input_info["original_label"]))
print("\n\n")
min_nPIR_mlwe = -1
max_nPIR_mlwe = 1

for le in sorted(mlwe_local_explanations_comb, key=lambda le: le["nPIR_original_top_class"], reverse=True ):
    if le["nPIR_original_top_class"] >= min_nPIR_mlwe and le["nPIR_original_top_class"] <= max_nPIR_mlwe:
        print_local_explanation(le, input_positions_tokens)
        

Original Probabilities: [0.9990951418876648, 0.0009048581123352051]
Original label: 0



LocalExplanationId: 41, FeatureId: 3
feature type:  MLWE
feature description:  Combination of Cluster 0 and Cluster 1
feature (position,token):  {'2': 'this', '3': 'movie', '9': 'just', '10': 'for', '11': 'kicks', '12': 'decided', '13': 'to', '14': 'rent', '15': 'it', '16': 'and', '17': 'watch', '18': 'it', '21': 'the', '22': 'plot', '23': 'is', '24': 'a', '25': 'carbon', '26': 'copy', '27': 'from', '28': 'fright', '31': 'i', '32': 'did', '33': 'like', '43': 'otherwise', '44': 'it', '45': 'was', '46': 'not', '47': 'good', '48': 'at', '49': 'all', '29': 'night', '34': 'the', '35': 'hairy', '36': 'vampire', '37': 'and', '38': 'the', '39': 'bug', '40': 'eating', '41': 'driver'}
Perturbed Text:   i saw many years ago, and again....
perturbed probabilities:  [0.004752695560455322, 0.9952473044395447]
nPIR Original Class:  0.995






LocalExplanationId: 38, FeatureId: 0
feature type:  MLWE
feature description:  Cluster 0
feature (position,token):  {'2': 'this', '3': 'movie', '9': 'just', '10': 'for', '11': 'kicks', '12': 'decided', '13': 'to', '14': 'rent', '15': 'it', '16': 'and', '17': 'watch', '18': 'it', '21': 'the', '22': 'plot', '23': 'is', '24': 'a', '25': 'carbon', '26': 'copy', '27': 'from', '28': 'fright', '31': 'i', '32': 'did', '33': 'like', '43': 'otherwise', '44': 'it', '45': 'was', '46': 'not', '47': 'good', '48': 'at', '49': 'all'}
Perturbed Text:   i saw many years ago, and again. night. the hairy vampire and the bug eating driver..
perturbed probabilities:  [0.07169586420059204, 0.928304135799408]
nPIR Original Class:  0.929






LocalExplanationId: 42, FeatureId: 4
feature type:  MLWE
feature description:  Combination of Cluster 0 and Cluster 2
feature (position,token):  {'2': 'this', '3': 'movie', '9': 'just', '10': 'for', '11': 'kicks', '12': 'decided', '13': 'to', '14': 'rent', '15': 'it', '16': 'and', '17': 'watch', '18': 'it', '21': 'the', '22': 'plot', '23': 'is', '24': 'a', '25': 'carbon', '26': 'copy', '27': 'from', '28': 'fright', '31': 'i', '32': 'did', '33': 'like', '43': 'otherwise', '44': 'it', '45': 'was', '46': 'not', '47': 'good', '48': 'at', '49': 'all', '0': 'i', '1': 'saw', '4': 'many', '5': 'years', '6': 'ago', '7': ',', '8': 'and', '19': 'again', '20': '.', '30': '.', '42': '.', '50': '.'}
Perturbed Text:   night the hairy vampire and the bug eating driver
perturbed probabilities:  [0.5541527271270752, 0.4458473026752472]
nPIR Original Class:  0.512






LocalExplanationId: 43, FeatureId: 5
feature type:  MLWE
feature description:  Combination of Cluster 1 and Cluster 2
feature (position,token):  {'29': 'night', '34': 'the', '35': 'hairy', '36': 'vampire', '37': 'and', '38': 'the', '39': 'bug', '40': 'eating', '41': 'driver', '0': 'i', '1': 'saw', '4': 'many', '5': 'years', '6': 'ago', '7': ',', '8': 'and', '19': 'again', '20': '.', '30': '.', '42': '.', '50': '.'}
Perturbed Text:   this movie just for kicks decided to rent it and watch it the plot is a carbon copy from fright i did like otherwise it was not good at all
perturbed probabilities:  [0.9985443353652954, 0.0014556348323822021]
nPIR Original Class:  0.001






LocalExplanationId: 40, FeatureId: 2
feature type:  MLWE
feature description:  Cluster 2
feature (position,token):  {'0': 'i', '1': 'saw', '4': 'many', '5': 'years', '6': 'ago', '7': ',', '8': 'and', '19': 'again', '20': '.', '30': '.', '42': '.', '50': '.'}
Perturbed Text:   this movie just for kicks decided to rent it and watch it the plot is a carbon copy from fright night i did like the hairy vampire and the bug eating driver otherwise it was not good at all
perturbed probabilities:  [0.9987757802009583, 0.001224219799041748]
nPIR Original Class:  0.001






LocalExplanationId: 39, FeatureId: 1
feature type:  MLWE
feature description:  Cluster 1
feature (position,token):  {'29': 'night', '34': 'the', '35': 'hairy', '36': 'vampire', '37': 'and', '38': 'the', '39': 'bug', '40': 'eating', '41': 'driver'}
Perturbed Text:   i saw this movie many years ago, and just for kicks decided to rent it and watch it again. the plot is a carbon copy from fright. i did like. otherwise it was not good at all.
perturbed probabilities:  [0.999262809753418, 0.0007371902465820312]
nPIR Original Class:  -0.0


In [12]:
display(HTML('<h1>Parts-of-speech Features Summary</h1>'))
display(HTML('<b>Predicted Class: {} </b>'.format(label_name(input_info["original_label"]))))
display(HTML('<b>Original Probability: {} </b>'.format(input_info["original_probabilities"])))

html_string = get_summary_string(pos_positions_tokens_score)
display(HTML(html_string))

display(HTML('(Features are highlighted in green if are positively influential for the predicted class, red otherwise. The intensity of the color is proportional to the nPIR score obtained by the word.) '))  


# POS Most influential feature

In [13]:
most_influnetial_pos_feature =max(pos_local_explanations_comb, key= lambda le: le["nPIR_original_top_class"])
print_local_explanation(most_influnetial_pos_feature,input_positions_tokens)

LocalExplanationId: 14, FeatureId: 14
feature type:  POS
feature description:  Combination of Nouns and Adverbs, Modals
feature (position,token):  {'0': 'i', '3': 'movie', '5': 'years', '11': 'kicks', '22': 'plot', '25': 'carbon', '26': 'copy', '29': 'night', '31': 'i', '35': 'hairy', '36': 'vampire', '39': 'bug', '41': 'driver', '6': 'ago', '9': 'just', '19': 'again', '46': 'not'}
Perturbed Text:   saw this many, and for decided to rent it and watch it. the is a from fright. did like the and the eating. otherwise it was good at all.
perturbed probabilities:  [0.003729104995727539, 0.9962708950042725]
nPIR Original Class:  0.996


<h1>Query Parts-of-speech Features</h1>

In [14]:
print("Original Probabilities: {}".format(input_info["original_probabilities"]))
print("Original label: {}".format(input_info["original_label"]))
print("\n\n")
min_nPIR_pos = -1
max_nPIR_pos = 1

for le in sorted(pos_local_explanations_comb, key=lambda le: le["nPIR_original_top_class"], reverse=True ):
    if le["nPIR_original_top_class"] >= min_nPIR_pos and le["nPIR_original_top_class"] <= max_nPIR_pos:
        print_local_explanation(le, input_positions_tokens)

Original Probabilities: [0.9990951418876648, 0.0009048581123352051]
Original label: 0



LocalExplanationId: 14, FeatureId: 14
feature type:  POS
feature description:  Combination of Nouns and Adverbs, Modals
feature (position,token):  {'0': 'i', '3': 'movie', '5': 'years', '11': 'kicks', '22': 'plot', '25': 'carbon', '26': 'copy', '29': 'night', '31': 'i', '35': 'hairy', '36': 'vampire', '39': 'bug', '41': 'driver', '6': 'ago', '9': 'just', '19': 'again', '46': 'not'}
Perturbed Text:   saw this many, and for decided to rent it and watch it. the is a from fright. did like the and the eating. otherwise it was good at all.
perturbed probabilities:  [0.003729104995727539, 0.9962708950042725]
nPIR Original Class:  0.996






LocalExplanationId: 18, FeatureId: 18
feature type:  POS
feature description:  Combination of Verbs and Adverbs, Modals
feature (position,token):  {'1': 'saw', '12': 'decided', '14': 'rent', '17': 'watch', '23': 'is', '32': 'did', '40': 'eating', '43': 'otherwise', '45': 'was', '6': 'ago', '9': 'just', '19': 'again', '46': 'not'}
Perturbed Text:   i this movie many years, and for kicks to it and it. the plot a carbon copy from fright night. i like the hairy vampire and the bug driver. it good at all.
perturbed probabilities:  [0.004296004772186279, 0.9957039952278137]
nPIR Original Class:  0.996






LocalExplanationId: 3, FeatureId: 3
feature type:  POS
feature description:  Adverbs, Modals
feature (position,token):  {'6': 'ago', '9': 'just', '19': 'again', '46': 'not'}
Perturbed Text:   i saw this movie many years, and for kicks decided to rent it and watch it. the plot is a carbon copy from fright night. i did like the hairy vampire and the bug eating driver. otherwise it was good at all.
perturbed probabilities:  [0.0650288462638855, 0.9349711537361145]
nPIR Original Class:  0.935






LocalExplanationId: 23, FeatureId: 23
feature type:  POS
feature description:  Combination of Adverbs, Modals and Interjections
feature (position,token):  {'6': 'ago', '9': 'just', '19': 'again', '46': 'not'}
Perturbed Text:   i saw this movie many years, and for kicks decided to rent it and watch it. the plot is a carbon copy from fright night. i did like the hairy vampire and the bug eating driver. otherwise it was good at all.
perturbed probabilities:  [0.0650288462638855, 0.9349711537361145]
nPIR Original Class:  0.935






LocalExplanationId: 24, FeatureId: 24
feature type:  POS
feature description:  Combination of Adverbs, Modals and Others (Cardinal Numbers, Foreign Words, Symbols)
feature (position,token):  {'6': 'ago', '9': 'just', '19': 'again', '46': 'not'}
Perturbed Text:   i saw this movie many years, and for kicks decided to rent it and watch it. the plot is a carbon copy from fright night. i did like the hairy vampire and the bug eating driver. otherwise it was good at all.
perturbed probabilities:  [0.0650288462638855, 0.9349711537361145]
nPIR Original Class:  0.935


...





LocalExplanationId: 26, FeatureId: 26
feature type:  POS
feature description:  Combination of Pronouns, Predeterminers, WH and Others (Cardinal Numbers, Foreign Words, Symbols)
feature (position,token):  {'15': 'it', '18': 'it', '44': 'it'}
Perturbed Text:   i saw this movie many years ago, and just for kicks decided to rent and watch again. the plot is a carbon copy from fright night. i did like the hairy vampire and the bug eating driver. otherwise was not good at all.
perturbed probabilities:  [0.9987318515777588, 0.0012681186199188232]
nPIR Original Class:  0.001






LocalExplanationId: 2, FeatureId: 2
feature type:  POS
feature description:  Verbs
feature (position,token):  {'1': 'saw', '12': 'decided', '14': 'rent', '17': 'watch', '23': 'is', '32': 'did', '40': 'eating', '43': 'otherwise', '45': 'was'}
Perturbed Text:   i this movie many years ago, and just for kicks to it and it again. the plot a carbon copy from fright night. i like the hairy vampire and the bug driver. it not good at all.
perturbed probabilities:  [0.9994001984596252, 0.0005998015403747559]
nPIR Original Class:  -0.001






LocalExplanationId: 20, FeatureId: 20
feature type:  POS
feature description:  Combination of Verbs and Interjections
feature (position,token):  {'1': 'saw', '12': 'decided', '14': 'rent', '17': 'watch', '23': 'is', '32': 'did', '40': 'eating', '43': 'otherwise', '45': 'was'}
Perturbed Text:   i this movie many years ago, and just for kicks to it and it again. the plot a carbon copy from fright night. i like the hairy vampire and the bug driver. it not good at all.
perturbed probabilities:  [0.9994001984596252, 0.0005998015403747559]
nPIR Original Class:  -0.001






LocalExplanationId: 21, FeatureId: 21
feature type:  POS
feature description:  Combination of Verbs and Others (Cardinal Numbers, Foreign Words, Symbols)
feature (position,token):  {'1': 'saw', '12': 'decided', '14': 'rent', '17': 'watch', '23': 'is', '32': 'did', '40': 'eating', '43': 'otherwise', '45': 'was'}
Perturbed Text:   i this movie many years ago, and just for kicks to it and it again. the plot a carbon copy from fright night. i like the hairy vampire and the bug driver. it not good at all.
perturbed probabilities:  [0.9994001984596252, 0.0005998015403747559]
nPIR Original Class:  -0.001






LocalExplanationId: 19, FeatureId: 19
feature type:  POS
feature description:  Combination of Verbs and Pronouns, Predeterminers, WH
feature (position,token):  {'1': 'saw', '12': 'decided', '14': 'rent', '17': 'watch', '23': 'is', '32': 'did', '40': 'eating', '43': 'otherwise', '45': 'was', '15': 'it', '18': 'it', '44': 'it'}
Perturbed Text:   i this movie many years ago, and just for kicks to and again. the plot a carbon copy from fright night. i like the hairy vampire and the bug driver. not good at all.
perturbed probabilities:  [0.9994635581970215, 0.0005364716053009033]
nPIR Original Class:  -0.001


In [15]:
display(HTML('<h1>Sentences Features Summary</h1>'))
display(HTML('<b>Predicted Class: {} </b>'.format(label_name(input_info["original_label"]))))
display(HTML('<b>Original Probability: {} </b>'.format(input_info["original_probabilities"])))

html_sen_string = get_summary_string(sen_positions_tokens_score)
        
display(HTML(html_sen_string))

display(HTML('(Features are highlighted in green if are positively influential for the predicted class, red otherwise. The intensity of the color is proportional to the nPIR score obtained by the word.) '))  


# SEN Most influential feature

In [16]:
most_influnetial_sen_feature =max(sen_local_explanations_comb, key= lambda le: le["nPIR_original_top_class"])
print_local_explanation(most_influnetial_sen_feature,input_positions_tokens)

LocalExplanationId: 36, FeatureId: 8
feature type:  SEN
feature description:  Combination of 2° Sentence and 4° Sentence
feature (position,token):  {'21': 'the', '22': 'plot', '23': 'is', '24': 'a', '25': 'carbon', '26': 'copy', '27': 'from', '28': 'fright', '29': 'night', '30': '.', '43': 'otherwise', '44': 'it', '45': 'was', '46': 'not', '47': 'good', '48': 'at', '49': 'all', '50': '.'}
Perturbed Text:   i saw this movie many years ago, and just for kicks decided to rent it and watch it again. i did like the hairy vampire and the bug eating driver.
perturbed probabilities:  [0.009644746780395508, 0.9903552532196045]
nPIR Original Class:  0.99


<h1> Query Sentences Features</h1>

In [17]:
print("Original Probabilities: {}".format(input_info["original_probabilities"]))
print("Original label: {}".format(input_info["original_label"]))
print("\n\n")
min_nPIR_sen = -1
max_nPIR_sen = 1

for le in sorted(sen_local_explanations_comb, key=lambda le: le["nPIR_original_top_class"], reverse=True ):
    if le["nPIR_original_top_class"] >= min_nPIR_sen and le["nPIR_original_top_class"] <= max_nPIR_sen:
        print_local_explanation(le, input_positions_tokens)

Original Probabilities: [0.9990951418876648, 0.0009048581123352051]
Original label: 0



LocalExplanationId: 36, FeatureId: 8
feature type:  SEN
feature description:  Combination of 2° Sentence and 4° Sentence
feature (position,token):  {'21': 'the', '22': 'plot', '23': 'is', '24': 'a', '25': 'carbon', '26': 'copy', '27': 'from', '28': 'fright', '29': 'night', '30': '.', '43': 'otherwise', '44': 'it', '45': 'was', '46': 'not', '47': 'good', '48': 'at', '49': 'all', '50': '.'}
Perturbed Text:   i saw this movie many years ago, and just for kicks decided to rent it and watch it again. i did like the hairy vampire and the bug eating driver.
perturbed probabilities:  [0.009644746780395508, 0.9903552532196045]
nPIR Original Class:  0.99






LocalExplanationId: 37, FeatureId: 9
feature type:  SEN
feature description:  Combination of 3° Sentence and 4° Sentence
feature (position,token):  {'31': 'i', '32': 'did', '33': 'like', '34': 'the', '35': 'hairy', '36': 'vampire', '37': 'and', '38': 'the', '39': 'bug', '40': 'eating', '41': 'driver', '42': '.', '43': 'otherwise', '44': 'it', '45': 'was', '46': 'not', '47': 'good', '48': 'at', '49': 'all', '50': '.'}
Perturbed Text:   i saw this movie many years ago, and just for kicks decided to rent it and watch it again. the plot is a carbon copy from fright night.
perturbed probabilities:  [0.608443558216095, 0.39155644178390503]
nPIR Original Class:  0.468






LocalExplanationId: 31, FeatureId: 3
feature type:  SEN
feature description:  4° Sentence
feature (position,token):  {'43': 'otherwise', '44': 'it', '45': 'was', '46': 'not', '47': 'good', '48': 'at', '49': 'all', '50': '.'}
Perturbed Text:   i saw this movie many years ago, and just for kicks decided to rent it and watch it again. the plot is a carbon copy from fright night. i did like the hairy vampire and the bug eating driver.
perturbed probabilities:  [0.6475638151168823, 0.3524361848831177]
nPIR Original Class:  0.435






LocalExplanationId: 34, FeatureId: 6
feature type:  SEN
feature description:  Combination of 1° Sentence and 4° Sentence
feature (position,token):  {'0': 'i', '1': 'saw', '2': 'this', '3': 'movie', '4': 'many', '5': 'years', '6': 'ago', '7': ',', '8': 'and', '9': 'just', '10': 'for', '11': 'kicks', '12': 'decided', '13': 'to', '14': 'rent', '15': 'it', '16': 'and', '17': 'watch', '18': 'it', '19': 'again', '20': '.', '43': 'otherwise', '44': 'it', '45': 'was', '46': 'not', '47': 'good', '48': 'at', '49': 'all', '50': '.'}
Perturbed Text:   the plot is a carbon copy from fright night. i did like the hairy vampire and the bug eating driver.
perturbed probabilities:  [0.6767352819442749, 0.3232647180557251]
nPIR Original Class:  0.41






LocalExplanationId: 32, FeatureId: 4
feature type:  SEN
feature description:  Combination of 1° Sentence and 2° Sentence
feature (position,token):  {'0': 'i', '1': 'saw', '2': 'this', '3': 'movie', '4': 'many', '5': 'years', '6': 'ago', '7': ',', '8': 'and', '9': 'just', '10': 'for', '11': 'kicks', '12': 'decided', '13': 'to', '14': 'rent', '15': 'it', '16': 'and', '17': 'watch', '18': 'it', '19': 'again', '20': '.', '21': 'the', '22': 'plot', '23': 'is', '24': 'a', '25': 'carbon', '26': 'copy', '27': 'from', '28': 'fright', '29': 'night', '30': '.'}
Perturbed Text:   i did like the hairy vampire and the bug eating driver. otherwise it was not good at all.
perturbed probabilities:  [0.9869811534881592, 0.013018876314163208]
nPIR Original Class:  0.024






LocalExplanationId: 29, FeatureId: 1
feature type:  SEN
feature description:  2° Sentence
feature (position,token):  {'21': 'the', '22': 'plot', '23': 'is', '24': 'a', '25': 'carbon', '26': 'copy', '27': 'from', '28': 'fright', '29': 'night', '30': '.'}
Perturbed Text:   i saw this movie many years ago, and just for kicks decided to rent it and watch it again. i did like the hairy vampire and the bug eating driver. otherwise it was not good at all.
perturbed probabilities:  [0.9988975524902344, 0.0011024177074432373]
nPIR Original Class:  0.0






LocalExplanationId: 35, FeatureId: 7
feature type:  SEN
feature description:  Combination of 2° Sentence and 3° Sentence
feature (position,token):  {'21': 'the', '22': 'plot', '23': 'is', '24': 'a', '25': 'carbon', '26': 'copy', '27': 'from', '28': 'fright', '29': 'night', '30': '.', '31': 'i', '32': 'did', '33': 'like', '34': 'the', '35': 'hairy', '36': 'vampire', '37': 'and', '38': 'the', '39': 'bug', '40': 'eating', '41': 'driver', '42': '.'}
Perturbed Text:   i saw this movie many years ago, and just for kicks decided to rent it and watch it again. otherwise it was not good at all.
perturbed probabilities:  [0.998989462852478, 0.001010507345199585]
nPIR Original Class:  0.0






LocalExplanationId: 33, FeatureId: 5
feature type:  SEN
feature description:  Combination of 1° Sentence and 3° Sentence
feature (position,token):  {'0': 'i', '1': 'saw', '2': 'this', '3': 'movie', '4': 'many', '5': 'years', '6': 'ago', '7': ',', '8': 'and', '9': 'just', '10': 'for', '11': 'kicks', '12': 'decided', '13': 'to', '14': 'rent', '15': 'it', '16': 'and', '17': 'watch', '18': 'it', '19': 'again', '20': '.', '31': 'i', '32': 'did', '33': 'like', '34': 'the', '35': 'hairy', '36': 'vampire', '37': 'and', '38': 'the', '39': 'bug', '40': 'eating', '41': 'driver', '42': '.'}
Perturbed Text:   the plot is a carbon copy from fright night. otherwise it was not good at all.
perturbed probabilities:  [0.9991543292999268, 0.0008456707000732422]
nPIR Original Class:  -0.0






LocalExplanationId: 30, FeatureId: 2
feature type:  SEN
feature description:  3° Sentence
feature (position,token):  {'31': 'i', '32': 'did', '33': 'like', '34': 'the', '35': 'hairy', '36': 'vampire', '37': 'and', '38': 'the', '39': 'bug', '40': 'eating', '41': 'driver', '42': '.'}
Perturbed Text:   i saw this movie many years ago, and just for kicks decided to rent it and watch it again. the plot is a carbon copy from fright night. otherwise it was not good at all.
perturbed probabilities:  [0.9993358254432678, 0.0006641745567321777]
nPIR Original Class:  -0.0






LocalExplanationId: 28, FeatureId: 0
feature type:  SEN
feature description:  1° Sentence
feature (position,token):  {'0': 'i', '1': 'saw', '2': 'this', '3': 'movie', '4': 'many', '5': 'years', '6': 'ago', '7': ',', '8': 'and', '9': 'just', '10': 'for', '11': 'kicks', '12': 'decided', '13': 'to', '14': 'rent', '15': 'it', '16': 'and', '17': 'watch', '18': 'it', '19': 'again', '20': '.'}
Perturbed Text:   the plot is a carbon copy from fright night. i did like the hairy vampire and the bug eating driver. otherwise it was not good at all.
perturbed probabilities:  [0.9994235038757324, 0.0005764961242675781]
nPIR Original Class:  -0.001
